# **Dependancies**

In [184]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import *
from keras.callbacks import *
import os
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from commons import mean_absolute_percentage_error
from keras.layers import *
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.losses import LogCosh

# **Loading Data**

In [185]:
PATH_TO_DATA = "reg_interval20-23nd.csv"
data = pd.read_csv(PATH_TO_DATA)

In [186]:
data.head()

,fee_to_rewardUSD,mediantransactionvalue30trxUSD,mining_profitability90trx,price30varUSD,price3momUSD,price3varUSD,price90momUSD,price90varUSD,sentinusdUSD,top100cap3trx,transactionvalueUSD,priceUSD
0,0.457,-0.240,-0.307,28623.0,-231.566,3878.0,-1017.000,573722.0,2.022007e+09,0.003,12757.0,7127.0
1,0.776,-0.261,-0.308,29355.0,-261.045,4729.0,-1046.000,581575.0,3.495276e+09,-0.020,16735.0,7250.0
2,0.770,-0.279,-0.310,29328.0,-42.402,3051.0,-878.841,587195.0,5.664998e+09,-0.020,23302.0,7390.0
3,0.583,-0.300,-0.311,29574.0,152.469,11526.0,-626.347,590895.0,3.470998e+09,-0.010,16928.0,7503.0
4,0.580,-0.325,-0.312,30437.0,376.272,10731.0,-565.003,593111.0,2.540099e+09,0.003,14695.0,7588.0


In [187]:
data.shape

(1097, 12)

In [188]:
y = data['priceUSD']

In [189]:
X = data

In [190]:
X['priceUSD'] = X['priceUSD'].shift(1,fill_value=12255)

In [191]:
X

,fee_to_rewardUSD,mediantransactionvalue30trxUSD,mining_profitability90trx,price30varUSD,price3momUSD,price3varUSD,price90momUSD,price90varUSD,sentinusdUSD,top100cap3trx,transactionvalueUSD,priceUSD
0,0.457,-0.240,-0.307,28623.0,-231.566,3878.000,-1017.000,573722.0,2.022007e+09,0.003,12757.0,12255.0
1,0.776,-0.261,-0.308,29355.0,-261.045,4729.000,-1046.000,581575.0,3.495276e+09,-0.020,16735.0,7127.0
2,0.770,-0.279,-0.310,29328.0,-42.402,3051.000,-878.841,587195.0,5.664998e+09,-0.020,23302.0,7250.0
3,0.583,-0.300,-0.311,29574.0,152.469,11526.000,-626.347,590895.0,3.470998e+09,-0.010,16928.0,7390.0
4,0.580,-0.325,-0.312,30437.0,376.272,10731.000,-565.003,593111.0,2.540099e+09,0.003,14695.0,7503.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1092,1.932,-0.085,-0.406,82135.0,-179.890,8243.000,-2758.000,2474152.0,5.572254e+09,0.031,53190.0,16640.0
1093,2.283,-0.087,-0.405,76543.0,-258.919,8530.000,-2936.000,2479394.0,6.313827e+09,0.047,54195.0,16591.0
1094,2.161,-0.090,-0.404,82274.0,-257.127,1350.000,-2778.000,2489811.0,7.554070e+09,0.027,50405.0,16550.0
1095,1.235,-0.094,-0.403,86919.0,-69.693,287.999,-2661.000,2500289.0,4.493923e+09,0.030,42790.0,16570.0


In [192]:
#X1 = data
#X1['target'] = X1['priceUSD'].shift(-1)


In [193]:
#X1.dropna(subset=['target'], inplace=True)

In [194]:
#X = data.iloc[:,:-1]

In [195]:
#y=data.iloc[:,-1:]

In [196]:
#y = data['priceUSD'].shift(-1,fill_value=1)

In [197]:
#y = data['priceUSD']

In [198]:
y

0        7127.0
1        7250.0
2        7390.0
3        7503.0
4        7588.0
         ...   
1092    16591.0
1093    16550.0
1094    16570.0
1095    16559.0
1096    16691.0
Name: priceUSD, Length: 1097, dtype: float64

In [199]:
def create_dataset(dataset, time_step=1):
    dataX = []
    datay = []
    for i in range(len(dataset) - time_step):
        a = dataset.iloc[i:(i + time_step), :].values  # Use .iloc for DataFrame
        dataX.append(a)
        datay.append(dataset.iloc[i + time_step, -1])
    return np.array(dataX), np.array(datay)


In [200]:
#time_step = 15
#X2 ,y2 = create_dataset(X1, time_step)

#print("X: ", X2.shape)
#print("y: ", y2.shape)

In [201]:
#X2

In [202]:
#y2

In [203]:
#y2_reshaped = y2.reshape(-1, 1, 1)

In [204]:
#y2_reshaped

In [205]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.2, train_size=0.8, shuffle=False, random_state=8)

In [206]:
X_train.shape

(877, 12)

In [207]:
estimators=[]

In [208]:
estimators.append(['robust',RobustScaler()])

In [209]:
estimators.append(['mixmax',MinMaxScaler()])

In [210]:
scale=Pipeline(estimators,verbose=True)

In [211]:
X_train=scale.fit_transform(X_train)

[Pipeline] ............ (step 1 of 2) Processing robust, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing mixmax, total=   0.0s


In [212]:
X_test=scale.transform(X_test)

In [213]:
X_train=np.reshape(X_train,(X_train.shape[0],1,X_train.shape[1]))

In [214]:
X_test=np.reshape(X_test,(X_test.shape[0],1,X_test.shape[1]))

In [215]:
y_train=y_train.values

In [216]:
y_train=np.reshape(y_train, (y_train.shape[0],1,1))

In [217]:
y_test=y_test.values

In [218]:
y_test=np.reshape(y_test,(y_test.shape[0],1,1))

In [219]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [220]:
X_train.shape

(877, 1, 12)

In [221]:
X_train

array([[[0.        , 0.1867065 , 0.03035714, ..., 0.40980393,
         0.        , 0.1159221 ]],

       [[0.01123754, 0.17102315, 0.02946429, ..., 0.39477125,
         0.00163186, 0.0339292 ]],

       [[0.01102617, 0.15758029, 0.02767857, ..., 0.39477125,
         0.00432579, 0.03589588]],

       ...,

       [[0.05830133, 0.44660193, 0.03214286, ..., 0.4130719 ,
         0.1255354 , 0.38865083]],

       [[0.0585127 , 0.44436148, 0.03035714, ..., 0.43333334,
         0.07643056, 0.39578202]],

       [[0.05301723, 0.44286782, 0.02857143, ..., 0.43986928,
         0.10168917, 0.39096928]]], dtype=float32)

In [222]:
y_train.shape

(877, 1, 1)

In [223]:
y_train

array([[[ 7127.]],

       [[ 7250.]],

       [[ 7390.]],

       [[ 7503.]],

       [[ 7588.]],

       [[ 7953.]],

       [[ 8273.]],

       [[ 7943.]],

       [[ 7933.]],

       [[ 8120.]],

       [[ 8146.]],

       [[ 8151.]],

       [[ 8605.]],

       [[ 8778.]],

       [[ 8693.]],

       [[ 8869.]],

       [[ 8907.]],

       [[ 8859.]],

       [[ 8670.]],

       [[ 8672.]],

       [[ 8693.]],

       [[ 8469.]],

       [[ 8410.]],

       [[ 8349.]],

       [[ 8466.]],

       [[ 8744.]],

       [[ 9049.]],

       [[ 9349.]],

       [[ 9394.]],

       [[ 9366.]],

       [[ 9393.]],

       [[ 9398.]],

       [[ 9356.]],

       [[ 9233.]],

       [[ 9421.]],

       [[ 9706.]],

       [[ 9798.]],

       [[ 9823.]],

       [[10083.]],

       [[ 9932.]],

       [[ 9968.]],

       [[10331.]],

       [[10289.]],

       [[10272.]],

       [[10152.]],

       [[ 9916.]],

       [[ 9723.]],

       [[ 9840.]],

       [[10091.]],

       [[ 9612.]],



In [224]:
#X_train, X_test, y_train, y_test =train_test_split(X2,y2_reshaped, test_size=0.2, train_size=0.8, shuffle=False, random_state=8)

In [225]:
#X_train.shape[2]

In [226]:
#y_train.shape

In [227]:
#X_test.shape

# **Model Architecture + Training**

In [228]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [229]:

adam = optimizers.Adam(learning_rate=lr_schedule(0), amsgrad=True)

Learning rate:  0.001


In [230]:
from tensorflow.keras import backend as K

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [231]:
model = Sequential()
model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu'), input_shape=(1, X_train.shape[2])))
model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu')))
model.add(Dense(1))
model.compile(loss="log_cosh", optimizer=adam, metrics=['mae'])

E:\DS440proj\BTCpred\.venv\lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [232]:
mcp_save = ModelCheckpoint('LSTM_reg_18-21rmse355.keras', save_best_only=True, monitor='val_loss', mode='auto')
earlyStopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='auto')

In [233]:
#model.compile(optimizer='adam', loss='log_cosh', metrics=['accuracy'])
model.fit(X_train,y_train, epochs=5000, batch_size=32, validation_data=(X_test,y_test), callbacks=[mcp_save,earlyStopping])

Epoch 1/5000
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - loss: 31110.5703 - mae: 31111.2637 - val_loss: 20485.0508 - val_mae: 20485.7441
Epoch 2/5000
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 31771.1934 - mae: 31771.8867 - val_loss: 20402.8809 - val_mae: 20403.5781
Epoch 3/5000
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 30596.3125 - mae: 30597.0039 - val_loss: 19994.1836 - val_mae: 19994.8789
Epoch 4/5000
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 29476.2148 - mae: 29476.9062 - val_loss: 18874.1152 - val_mae: 18874.8105
Epoch 5/5000
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 27475.4316 - mae: 27476.1250 - val_loss: 16339.9980 - val_mae: 16340.6904
Epoch 6/5000
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 21819.1895 - mae: 21819.8828 - val_loss: 11544.5488 - val_mae: 11545.2432
Epoch 7/5000
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 14015.1973 - mae: 14015.8896 - val_loss: 6207.8140 - val_mae: 6208.5078
Epoch 8/5000
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - lo

# **Testing Model**

In [234]:
#model.save('LSTM_complete_model344.h5', overwrite=True)

In [235]:
#from tensorflow.keras.models import load_model
#loaded_model = load_model('LSTM_complete_model344.h5')

In [236]:
y_pred = np.ravel(model.predict(X_test))

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


In [237]:
y_test=np.ravel(y_test)

In [238]:
y_pred

array([28962.758, 28909.299, 29178.941, 30571.6  , 31794.377, 31091.182,
       29843.54 , 29689.084, 29321.684, 29629.137, 31115.791, 29803.814,
       30312.514, 29988.918, 29525.484, 28713.5  , 27412.562, 24014.26 ,
       21927.283, 21358.734, 21434.518, 20679.97 , 19391.584, 19363.014,
       20344.055, 21034.309, 20350.678, 20564.588, 21155.748, 21321.615,
       21404.629, 21092.783, 20823.201, 20233.65 , 19385.719, 19560.254,
       19266.615, 19253.904, 19560.605, 20058.809, 20203.904, 20783.75 ,
       21786.047, 21646.395, 21136.707, 20483.613, 19651.742, 19582.754,
       20034.875, 20718.29 , 20852.105, 21207.031, 21792.465, 22464.576,
       23510.908, 22944.822, 23123.186, 22455.248, 22599.436, 21893.271,
       21017.66 , 21570.643, 23252.219, 23905.748, 24061.523, 23665.06 ,
       23152.46 , 22857.953, 23071.516, 22783.13 , 23044.684, 23139.63 ,
       23033.691, 23942.94 , 23548.604, 23574.705, 24467.572, 24046.754,
       24563.12 , 24488.89 , 24359.04 , 23991.805, 

In [239]:
r2=r2_score(y_test,y_pred) #testing score/ r^2
r2

0.9722489681030226

In [240]:
mae=mean_absolute_error(y_test,y_pred) #mae
mae

385.2131

In [241]:
rmse=np.sqrt(mean_squared_error(y_test,y_pred)) #rmse
rmse

570.2814

In [242]:
mape=mean_absolute_percentage_error(y_test,y_pred) #mape
mape

1.818205788731575

In [243]:
pd.DataFrame(zip(['MAE','RMSE','MAPE','R^2'],[mae,rmse,mape,r2])).transpose()

,0,1,2,3
0,MAE,RMSE,MAPE,R^2
1,385.213104,570.281372,1.818206,0.972249


In [244]:
pd.DataFrame([y_test,y_pred]).transpose()

,0,1
0,28888.0,28962.757812
1,29144.0,28909.298828
2,30589.0,29178.941406
3,31711.0,30571.599609
4,31096.0,31794.376953
...,...,...
215,16591.0,16661.167969
216,16550.0,16605.271484
217,16570.0,16558.369141
218,16559.0,16597.914062
